In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import re
import sqlite3

from nltk import word_tokenize 

In [2]:
pd.set_option('display.max_rows', 500, 'display.max_columns', 500)

In [3]:
sales = pd.read_csv('Iowa_Liquor_Sales.csv')

C:\Users\JeongwooHan\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (6,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [4]:
sales.head()

,Invoice/Item Number,Date,Store Number,Store Name,Address,City,Zip Code,Store Location,County Number,County,Category,Category Name,Vendor Number,Vendor Name,Item Number,Item Description,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons)
0,S31296100034,03/16/2016,3762,Wine and Spirits Gallery,7690 HICKMAN RD,WINDSOR HEIGHTS,50322.0,NaN,77.0,Polk,1081200.0,CREAM LIQUEURS,330.0,Gemini Spirits,80457,Ryan's Cream Liqueur,12,1000,7.23,10.85,1,10.85,1.0,0.26
1,INV-20599500022,07/15/2019,4320,Fareway Stores #470 / Perry,NaN,NaN,NaN,NaN,NaN,NaN,1012200.0,Scotch Whiskies,421.0,SAZERAC COMPANY INC,10008,Scoresby Rare Scotch,6,1750,10.50,15.75,6,94.50,10.5,2.77
2,INV-28403900139,07/01/2020,2643,Hy-Vee Wine and Spirits / Waterloo,2126 Kimball Ave,Waterloo,50701.0,NaN,7.0,BLACK HAWK,1012400.0,Irish Whiskies,370.0,PERNOD RICARD USA,15628,Jameson,6,1750,32.48,48.72,2,97.44,3.5,0.92
3,INV-20560900100,07/11/2019,3456,Quick Shop / Clear Lake,NaN,NaN,NaN,NaN,NaN,NaN,1012200.0,Scotch Whiskies,421.0,SAZERAC COMPANY INC,10009,Scoresby Rare Scotch,12,1000,6.74,10.11,2,20.22,2.0,0.52
4,INV-20784900043,07/23/2019,5151,IDA Liquor,"500, Hwy 175",Ida Grove,51445.0,NaN,47.0,IDA,1701100.0,Temporary & Specialty Packages,260.0,DIAGEO AMERICAS,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...,6,1850,18.00,27.00,6,162.00,11.1,2.93


In [5]:
sales.shape

(21641157, 24)

In [6]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21641157 entries, 0 to 21641156
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   Invoice/Item Number    object 
 1   Date                   object 
 2   Store Number           int64  
 3   Store Name             object 
 4   Address                object 
 5   City                   object 
 6   Zip Code               object 
 7   Store Location         object 
 8   County Number          float64
 9   County                 object 
 10  Category               float64
 11  Category Name          object 
 12  Vendor Number          float64
 13  Vendor Name            object 
 14  Item Number            object 
 15  Item Description       object 
 16  Pack                   int64  
 17  Bottle Volume (ml)     int64  
 18  State Bottle Cost      float64
 19  State Bottle Retail    float64
 20  Bottles Sold           int64  
 21  Sale (Dollars)         float64
 22  Volume Sold (Lit

In [7]:
sales.columns = sales.columns.str.replace(' ','')

In [8]:
sales.columns.values

array(['Invoice/ItemNumber', 'Date', 'StoreNumber', 'StoreName',
       'Address', 'City', 'ZipCode', 'StoreLocation', 'CountyNumber',
       'County', 'Category', 'CategoryName', 'VendorNumber', 'VendorName',
       'ItemNumber', 'ItemDescription', 'Pack', 'BottleVolume(ml)',
       'StateBottleCost', 'StateBottleRetail', 'BottlesSold',
       'Sale(Dollars)', 'VolumeSold(Liters)', 'VolumeSold(Gallons)'],
      dtype=object)

# Products Table

In [9]:
prod_col = ['Date',
           'Category',
           'CategoryName',
           'ItemNumber',
           'ItemDescription']

product = sales[prod_col].copy()

product.head()

,Date,Category,CategoryName,ItemNumber,ItemDescription
0,03/16/2016,1081200.0,CREAM LIQUEURS,80457,Ryan's Cream Liqueur
1,07/15/2019,1012200.0,Scotch Whiskies,10008,Scoresby Rare Scotch
2,07/01/2020,1012400.0,Irish Whiskies,15628,Jameson
3,07/11/2019,1012200.0,Scotch Whiskies,10009,Scoresby Rare Scotch
4,07/23/2019,1701100.0,Temporary & Specialty Packages,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...


In [10]:
product.shape

(21641157, 5)

In [11]:
product['ItemNumber'].nunique()

12363

In [12]:
product['ItemDescription'].nunique()

9497

In [13]:
product['CategoryName'].nunique()

134

In [14]:
product['Category'].nunique()

111

In [15]:
product.groupby(['Category', 'CategoryName']).agg({'Date': 'count'})

Date
Category  CategoryName                                 
1011100.0 BLENDED WHISKIES                       441610
          Blended Whiskies                       499204
1011200.0 STRAIGHT BOURBON WHISKIES              543684
          Straight Bourbon Whiskies              759581
1011250.0 SINGLE BARREL BOURBON WHISKIES           9268
1011300.0 Single Barrel Bourbon Whiskies          16136
          TENNESSEE WHISKIES                     245162
1011400.0 BOTTLED IN BOND BOURBON                  6417
          Tennessee Whiskies                     322660
1011500.0 Bottled in Bond Bourbon                  9611
          STRAIGHT RYE WHISKIES                   59869
1011600.0 CORN WHISKIES                             752
          Straight Rye Whiskies                   82175
1011700.0 Corn Whiskies                            7975
1011800.0 Iowa Distillery Whiskies                  124
1012000.0 Imported Whiskies                           7
1012100.0 CANADIAN WHISKIES                      936212
          Canadian Whiskies                     1153861
1012200.0 SCOTCH WHISKIES                        199770
          Scotch Whiskies                        191060
1012210.0 SINGLE MALT SCOTCH                      82675
1012300.0 IRISH WHISKIES                          88389
          Single Malt Scotch                     101219
1012400.0 Irish Whiskies                         146777
          JAPANESE WHISKY                           286
1022100.0 Mixto Tequila                          252961
          TEQUILA                                435298
1022200.0 100% Agave Tequila                     329889
1022300.0 Mezcal                                   6279
1031000.0 American Vodka                           5223
1031080.0 VODKA 80 PROOF                        1265974
1031090.0 OTHER PROOF VODKA                        3398
1031100.0 100 PROOF VODKA                         99124
          American Vodkas                       1747627
1031110.0 LOW PROOF VODKA                          1565
1031200.0 American Flavored Vodka                577235
          VODKA FLAVORED                         502813
1032000.0 Imported Vodka                           1542
1032080.0 IMPORTED VODKA                         404048
1032100.0 Imported Vodkas                        403706
1032200.0 IMPORTED VODKA - MISC                  256241
          Imported Flavored Vodka                233745
1032230.0 IMPORTED VODKA - CHERRY                    14
1041100.0 AMERICAN DRY GINS                      256386
          American Dry Gins                      235941
1041150.0 FLAVORED GINS                           12010
1041200.0 AMERICAN SLOE GINS                       7002
          Flavored Gin                             8711
1041300.0 American Sloe Gins                       3819
1042000.0 Imported Gins                               8
1042100.0 IMPORTED DRY GINS                      113313
          Imported Dry Gins                      126455
1051010.0 AMERICAN GRAPE BRANDIES                243144
1051100.0 APRICOT BRANDIES                        38509
          American Brandies                      374101
1051110.0 BLACKBERRY BRANDIES                     59714
1051120.0 CHERRY BRANDIES                         13216
1051140.0 PEACH BRANDIES                          15070
1051150.0 MISCELLANEOUS  BRANDIES                  4214
1052010.0 IMPORTED GRAPE BRANDIES                155688
1052100.0 Imported Brandies                      244689
1062050.0 BARBADOS RUM                             9909
1062100.0 Gold Rum                                74753
          JAMAICA RUM                             15288
1062200.0 PUERTO RICO & VIRGIN ISLANDS RUM       395376
          White Rum                              280408
1062300.0 Aged Dark Rum                           33077
          FLAVORED RUM                           278367
1062310.0 SPICED RUM                             530323
1062400.0 Spiced Rum                             568193
1062500.0 Flavored 

In [16]:
product['CategoryName'] = product['CategoryName'].str.lower()

In [17]:
product.groupby(['Category', 'CategoryName']).agg({'Date': 'count'})

Date
Category  CategoryName                                 
1011100.0 blended whiskies                       940814
1011200.0 straight bourbon whiskies             1303265
1011250.0 single barrel bourbon whiskies           9268
1011300.0 single barrel bourbon whiskies          16136
          tennessee whiskies                     245162
1011400.0 bottled in bond bourbon                  6417
          tennessee whiskies                     322660
1011500.0 bottled in bond bourbon                  9611
          straight rye whiskies                   59869
1011600.0 corn whiskies                             752
          straight rye whiskies                   82175
1011700.0 corn whiskies                            7975
1011800.0 iowa distillery whiskies                  124
1012000.0 imported whiskies                           7
1012100.0 canadian whiskies                     2090073
1012200.0 scotch whiskies                        390830
1012210.0 single malt scotch                      82675
1012300.0 irish whiskies                          88389
          single malt scotch                     101219
1012400.0 irish whiskies                         146777
          japanese whisky                           286
1022100.0 mixto tequila                          252961
          tequila                                435298
1022200.0 100% agave tequila                     329889
1022300.0 mezcal                                   6279
1031000.0 american vodka                           5223
1031080.0 vodka 80 proof                        1265974
1031090.0 other proof vodka                        3398
1031100.0 100 proof vodka                         99124
          american vodkas                       1747627
1031110.0 low proof vodka                          1565
1031200.0 american flavored vodka                577235
          vodka flavored                         502813
1032000.0 imported vodka                           1542
1032080.0 imported vodka                         404048
1032100.0 imported vodkas                        403706
1032200.0 imported flavored vodka                233745
          imported vodka - misc                  256241
1032230.0 imported vodka - cherry                    14
1041100.0 american dry gins                      492327
1041150.0 flavored gins                           12010
1041200.0 american sloe gins                       7002
          flavored gin                             8711
1041300.0 american sloe gins                       3819
1042000.0 imported gins                               8
1042100.0 imported dry gins                      239768
1051010.0 american grape brandies                243144
1051100.0 american brandies                      374101
          apricot brandies                        38509
1051110.0 blackberry brandies                     59714
1051120.0 cherry brandies                         13216
1051140.0 peach brandies                          15070
1051150.0 miscellaneous  brandies                  4214
1052010.0 imported grape brandies                155688
1052100.0 imported brandies                      244689
1062050.0 barbados rum                             9909
1062100.0 gold rum                                74753
          jamaica rum                             15288
1062200.0 puerto rico & virgin islands rum       395376
          white rum                              280408
1062300.0 aged dark rum                           33077
          flavored rum                           278367
1062310.0 spiced rum                             530323
1062400.0 spiced rum                             568193
1062500.0 flavored rum                           295568
1070000.0 cocktails / rtd                           948
1070100.0 temporary  & specialty packages            35
1071100.0 american cocktails                     284695
          cocktails /rtd                         334336
1081000.0 american cordials & liqueurs             2229
1081010.0 american 

In [18]:
product['Year'] = product['Date'].str[-4:]

In [19]:
product['Year'].value_counts()

2020    2614365
2019    2380345
2018    2355558
2017    2291276
2016    2279893
2015    2184483
2014    2097796
2012    2082059
2013    2063763
2021    1291619
Name: Year, dtype: int64

In [20]:
product[product['Category'] == 1701100.0].groupby(['Category', 'CategoryName','Year']).agg({'Date': 'count'})

Date
Category  CategoryName                   Year       
1701100.0 decanters & specialty packages 2012   6775
                                         2013   7421
                                         2014  12968
                                         2015  10577
                                         2016   2810
          temporary & specialty packages 2016   6358
                                         2017  14667
                                         2018  22907
                                         2019  34998
                                         2020  35458
                                         2021  13269

In [21]:
# Take in the most recent year's CategoryName and consolidate it with Category.

In [22]:
Category_List = product.sort_values(['Year']).drop_duplicates(subset = ['Category'], keep = 'last')

In [23]:
Category_List.loc[Category_List['Category'] == 1701100.0]

,Date,Category,CategoryName,ItemNumber,ItemDescription,Year
20780218,03/08/2021,1701100.0,temporary & specialty packages,67905,Baileys Colada,2021


In [24]:
Merged_Product = product.merge(Category_List[['Category', 'CategoryName']], on = 'Category', how = 'left')

In [25]:
Merged_Product.drop(columns = 'CategoryName_x', inplace = True)

In [26]:
Merged_Product.rename(columns = {'CategoryName_y': 'CategoryName'}, inplace = True)
Merged_Product

,Date,Category,ItemNumber,ItemDescription,Year,CategoryName
0,03/16/2016,1081200.0,80457,Ryan's Cream Liqueur,2016,cream liqueurs
1,07/15/2019,1012200.0,10008,Scoresby Rare Scotch,2019,scotch whiskies
2,07/01/2020,1012400.0,15628,Jameson,2020,irish whiskies
3,07/11/2019,1012200.0,10009,Scoresby Rare Scotch,2019,scotch whiskies
4,07/23/2019,1701100.0,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...,2019,temporary & specialty packages
...,...,...,...,...,...,...
21641152,06/29/2021,1032100.0,34007,Absolut Swedish Vodka 80prf,2021,imported vodkas
21641153,06/30/2021,1062500.0,72928,Captain Morgan Sliced Apple,2021,flavored rum
21641154,06/30/2021,1011200.0,17127,Cabin Still Str Bourbon,2021,straight bourbon whiskies
21641155,06/29/2021,1062400.0,43337,Captain Morgan Original Spiced,2021,spiced rum


In [27]:
Merged_Product.groupby(['Category', 'CategoryName']).agg({'Date': 'count'})

,,Date
Category,CategoryName,
1011100.0,blended whiskies,940814
1011200.0,straight bourbon whiskies,1303265
1011250.0,single barrel bourbon whiskies,9268
1011300.0,single barrel bourbon whiskies,261298
1011400.0,tennessee whiskies,329077
1011500.0,bottled in bond bourbon,69480
1011600.0,straight rye whiskies,82927
1011700.0,corn whiskies,8014
1011800.0,iowa distillery whiskies,124


In [28]:
product = Merged_Product.copy()

In [29]:
product.groupby(['ItemNumber', 'ItemDescription']).agg({'Date': 'count'})

,,Date
ItemNumber,ItemDescription,
101,Stolichnaya 80prf w/2 Stemless Martini Glasses,26
102,Bushmills 750ml w/Blackbush 50ml,34
104,Broker's Gin Holiday w/1 Collins Glass,15
107,Bushmills Black w/Rubber Ice Ball Molds,31
108,Midnight Moon Cran-Apple Pie,39
...,...,...
995097,Rondiaz DSS Lemon Rum,1
995206,HA Glen Garioch Virgin Oak,1
995539,Caribbean's Finest Rum Grapefruit,1


In [30]:
Item_List = product.sort_values(['Year']).drop_duplicates(subset = ['ItemNumber'], keep = 'last')

In [31]:
Merged_Item = product.merge(Item_List[['ItemNumber', 'ItemDescription']], on = 'ItemNumber', how = 'left')

In [32]:
Merged_Item.drop(columns = 'ItemDescription_x', inplace = True)

In [33]:
Merged_Item.rename(columns = {'ItemDescription_y': 'ItemDescription'}, inplace = True)
Merged_Item

,Date,Category,ItemNumber,Year,CategoryName,ItemDescription
0,03/16/2016,1081200.0,80457,2016,cream liqueurs,Ryans Cream Liqueur
1,07/15/2019,1012200.0,10008,2019,scotch whiskies,Scoresby Rare Scotch
2,07/01/2020,1012400.0,15628,2020,irish whiskies,Jameson
3,07/11/2019,1012200.0,10009,2019,scotch whiskies,Scoresby Rare Scotch
4,07/23/2019,1701100.0,100107,2019,temporary & specialty packages,Captain Morgan OSR Glass w/50ml Apple Smash & ...
...,...,...,...,...,...,...
21641152,06/29/2021,1032100.0,34007,2021,imported vodkas,Absolut Swedish Vodka 80prf
21641153,06/30/2021,1062500.0,72928,2021,flavored rum,Captain Morgan Sliced Apple
21641154,06/30/2021,1011200.0,17127,2021,straight bourbon whiskies,Cabin Still Str Bourbon
21641155,06/29/2021,1062400.0,43337,2021,spiced rum,Captain Morgan Original Spiced


In [34]:
product = Merged_Item.copy()

In [35]:
product = product[['ItemNumber', 'ItemDescription', 'Category', 'CategoryName']]

In [36]:
product.shape

(21641157, 4)

In [37]:
product.drop_duplicates(inplace = True)

In [38]:
product.shape

(15342, 4)

In [39]:
product.head()

,ItemNumber,ItemDescription,Category,CategoryName
0,80457,Ryans Cream Liqueur,1081200.0,cream liqueurs
1,10008,Scoresby Rare Scotch,1012200.0,scotch whiskies
2,15628,Jameson,1012400.0,irish whiskies
3,10009,Scoresby Rare Scotch,1012200.0,scotch whiskies
4,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...,1701100.0,temporary & specialty packages


# Product Price Table

In [40]:
prod_price_col = ['Date',
                  'ItemNumber',
                  'StateBottleRetail',
                  'BottlesSold',
                  'Sale(Dollars)']

product_price = sales[prod_price_col].copy()

product_price.head()

,Date,ItemNumber,StateBottleRetail,BottlesSold,Sale(Dollars)
0,03/16/2016,80457,10.85,1,10.85
1,07/15/2019,10008,15.75,6,94.50
2,07/01/2020,15628,48.72,2,97.44
3,07/11/2019,10009,10.11,2,20.22
4,07/23/2019,100107,27.00,6,162.00


In [41]:
product_price['Date'] = pd.to_datetime(product_price['Date'])

In [72]:
product_price.rename(columns = {'Sale(Dollars)': 'Sale'}, inplace = True)

# Vendor Table

In [43]:
vendor_col = ['Date',
              'VendorNumber',
              'VendorName']

vendor = sales[vendor_col].copy()

vendor.head()

,Date,VendorNumber,VendorName
0,03/16/2016,330.0,Gemini Spirits
1,07/15/2019,421.0,SAZERAC COMPANY INC
2,07/01/2020,370.0,PERNOD RICARD USA
3,07/11/2019,421.0,SAZERAC COMPANY INC
4,07/23/2019,260.0,DIAGEO AMERICAS


In [44]:
vendor.shape

(21641157, 3)

In [45]:
vendor['Year'] = vendor['Date'].str[-4:]

In [46]:
vendor

,Date,VendorNumber,VendorName,Year
0,03/16/2016,330.0,Gemini Spirits,2016
1,07/15/2019,421.0,SAZERAC COMPANY INC,2019
2,07/01/2020,370.0,PERNOD RICARD USA,2020
3,07/11/2019,421.0,SAZERAC COMPANY INC,2019
4,07/23/2019,260.0,DIAGEO AMERICAS,2019
...,...,...,...,...
21641152,06/29/2021,370.0,PERNOD RICARD USA,2021
21641153,06/30/2021,260.0,DIAGEO AMERICAS,2021
21641154,06/30/2021,259.0,Heaven Hill Brands,2021
21641155,06/29/2021,260.0,DIAGEO AMERICAS,2021


In [47]:
Vendor_List = vendor.sort_values(['Year']).drop_duplicates(subset = ['VendorNumber'], keep = 'last')

In [48]:
Merged_Vendor = vendor.merge(Vendor_List[['VendorNumber', 'VendorName']], on = 'VendorNumber', how = 'left')

In [49]:
Merged_Vendor.drop(columns = 'VendorName_x', inplace = True)

In [50]:
Merged_Vendor.rename(columns = {'VendorName_y': 'VendorName'}, inplace = True)
Merged_Vendor

,Date,VendorNumber,Year,VendorName
0,03/16/2016,330.0,2016,Gemini Spirits
1,07/15/2019,421.0,2019,SAZERAC COMPANY INC
2,07/01/2020,370.0,2020,PERNOD RICARD USA
3,07/11/2019,421.0,2019,SAZERAC COMPANY INC
4,07/23/2019,260.0,2019,DIAGEO AMERICAS
...,...,...,...,...
21641152,06/29/2021,370.0,2021,PERNOD RICARD USA
21641153,06/30/2021,260.0,2021,DIAGEO AMERICAS
21641154,06/30/2021,259.0,2021,Heaven Hill Brands
21641155,06/29/2021,260.0,2021,DIAGEO AMERICAS


In [51]:
vendor = Merged_Vendor[['VendorNumber', 'VendorName']]

In [52]:
vendor.drop_duplicates(inplace = True)

<ipython-input-52-5524dcceeb11>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vendor.drop_duplicates(inplace = True)


In [53]:
vendor.shape

(378, 2)

In [54]:
vendor.head()

,VendorNumber,VendorName
0,330.0,Gemini Spirits
1,421.0,SAZERAC COMPANY INC
2,370.0,PERNOD RICARD USA
4,260.0,DIAGEO AMERICAS
7,410.0,Patron Spirits Company


# Store Table

In [100]:
store_col = ['StoreNumber',
             'StoreName',
             'Address',
             'City',
             'ZipCode',
             'StoreLocation',
             'CountyNumber',
             'County']

store = sales[store_col].drop_duplicates(store_col, keep = 'first').copy()

store.dropna(inplace = True)

store.head()

,StoreNumber,StoreName,Address,City,ZipCode,StoreLocation,CountyNumber,County
9,2663,Hy-Vee Food Store / Urbandale,8701 Douglas,Urbandale,50322.0,POINT (-93.739105 41.629364),77.0,POLK
18,2647,Hy-Vee #7 / Cedar Rapids,5050 Edgewood Rd,Cedar Rapids,52411.0,POINT (-91.701581 42.030129),57.0,LINN
24,5259,Gameday Liquor,924 Locust St,Glenwood,51534.0,POINT (-95.916235 41.284951),65.0,MILLS
27,5597,Liquor Tobacco & Gas,1401 Central Ave,Dubuque,52001.0,POINT (-90.66784 42.50649),31.0,DUBUQUE
31,3749,Wal-Mart 1526 / Storm Lake,1831 Lake Ave NW,Storm Lake,50588.0,POINT (-95.200739 42.663904),11.0,BUENA VIST


In [101]:
store['StoreName'] = store['StoreName'].apply(lambda x: x.split('/')[0].strip())

In [102]:
def standardize_name(name):
    name = str(name)
    name = name.replace("``",'')
    name = name.replace('"','')
    name = re.sub("[,`'#.]",'',name)
    name = str(name).lower().replace(' and ',' & ')
    store_name = word_tokenize(name)
    store_name = [str(i).capitalize().strip() for i in store_name]
    store_name = ' '.join(store_name)
    store_name = store_name.replace('  ',' ')
    return store_name

In [103]:
for col in ['StoreName', 'City', 'County']:
    store[col] = store[col].apply(standardize_name)

In [104]:
def get_location(location):
    if str(location) != 'nan':
        location = str(location)
        location = re.findall(r'[-]\d.*\d', location)[0]
        lon, lat = location.split(' ')
        lon = round(float(lon), 2)
        lat = round(float(lat), 2)
        location = [lon, lat]
    else:
        location = np.nan
    return location

In [105]:
store['StoreLocation'] = store['StoreLocation'].map(get_location)

In [106]:
store

,StoreNumber,StoreName,Address,City,ZipCode,StoreLocation,CountyNumber,County
9,2663,Hy-vee Food Store,8701 Douglas,Urbandale,50322.0,"[-93.74, 41.63]",77.0,Polk
18,2647,Hy-vee 7,5050 Edgewood Rd,Cedar Rapids,52411.0,"[-91.7, 42.03]",57.0,Linn
24,5259,Gameday Liquor,924 Locust St,Glenwood,51534.0,"[-95.92, 41.28]",65.0,Mills
27,5597,Liquor Tobacco & Gas,1401 Central Ave,Dubuque,52001.0,"[-90.67, 42.51]",31.0,Dubuque
31,3749,Wal-mart 1526,1831 Lake Ave NW,Storm Lake,50588.0,"[-95.2, 42.66]",11.0,Buena Vist
...,...,...,...,...,...,...,...,...
21550279,6055,Caseys General Store 1428,426 N 2nd St,Milo,50166,"[-93.44, 41.29]",91.0,Warren
21570146,6172,Bp To Go,975 Robins Road,Hiawatha,52233,"[-91.67, 42.05]",57.0,Linn
21587940,6179,Bp To Go - 1010,1010 E Post Rd,Marion,52302,"[-91.6, 42.02]",57.0,Linn
21625528,6176,Mini-mart,1420 W 1st St,Cedar Falls,50613,"[-92.46, 42.54]",7.0,Black Hawk


In [171]:
store_test = store.copy()

In [172]:
long_lst = []
lat_lst = []
for i in store_test['StoreLocation']:
    long, lat = str(i).split(', ')
    long_lst.append(float(long.replace('[','')))
    lat_lst.append(float(lat.replace(']','')))
    
store_test['long'] = long_lst
store_test['lat'] = lat_lst
    

In [173]:
store_test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8710 entries, 9 to 21627782
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   StoreNumber    8710 non-null   int64  
 1   StoreName      8710 non-null   object 
 2   Address        8710 non-null   object 
 3   City           8710 non-null   object 
 4   ZipCode        8710 non-null   object 
 5   StoreLocation  8710 non-null   object 
 6   CountyNumber   8710 non-null   float64
 7   County         8710 non-null   object 
 8   long           8710 non-null   float64
 9   lat            8710 non-null   float64
dtypes: float64(3), int64(1), object(6)
memory usage: 1006.6+ KB


In [174]:
store_test.drop('StoreLocation', axis = 1, inplace = True)

In [175]:
store_test

,StoreNumber,StoreName,Address,City,ZipCode,CountyNumber,County,long,lat
9,2663,Hy-vee Food Store,8701 Douglas,Urbandale,50322.0,77.0,Polk,-93.74,41.63
18,2647,Hy-vee 7,5050 Edgewood Rd,Cedar Rapids,52411.0,57.0,Linn,-91.70,42.03
24,5259,Gameday Liquor,924 Locust St,Glenwood,51534.0,65.0,Mills,-95.92,41.28
27,5597,Liquor Tobacco & Gas,1401 Central Ave,Dubuque,52001.0,31.0,Dubuque,-90.67,42.51
31,3749,Wal-mart 1526,1831 Lake Ave NW,Storm Lake,50588.0,11.0,Buena Vist,-95.20,42.66
...,...,...,...,...,...,...,...,...,...
21550279,6055,Caseys General Store 1428,426 N 2nd St,Milo,50166,91.0,Warren,-93.44,41.29
21570146,6172,Bp To Go,975 Robins Road,Hiawatha,52233,57.0,Linn,-91.67,42.05
21587940,6179,Bp To Go - 1010,1010 E Post Rd,Marion,52302,57.0,Linn,-91.60,42.02
21625528,6176,Mini-mart,1420 W 1st St,Cedar Falls,50613,7.0,Black Hawk,-92.46,42.54


In [109]:
store_sql = store.drop('StoreLocation', axis = 1)

In [62]:
#store['HyVee_control'] = np.where(store['StoreName'].str.contains('Hy-vee'),1,0)

In [64]:
#store.loc[store['HyVee_control'] == 1]

,StoreNumber,StoreName,Address,City,ZipCode,StoreLocation,CountyNumber,County,HyVee_control
9,2663,Hy-vee Food Store,8701 Douglas,Urbandale,50322.0,"(-93.74, 41.63)",77.0,Polk,1
18,2647,Hy-vee 7,5050 Edgewood Rd,Cedar Rapids,52411.0,"(-91.7, 42.03)",57.0,Linn,1
34,2565,Hy-vee Food Store 1636,819 N Grand Ave,Spencer,51301.0,"(-95.15, 43.15)",21.0,Clay,1
61,2662,Hy-vee Wine & Spirits,"522 Mulberry, Suite A",Muscatine,52761.0,"(-91.04, 41.43)",70.0,Muscatine,1
64,2555,Hy-vee Food Store,3111 Main,Keokuk,52632.0,"(-91.4, 40.42)",56.0,Lee,1
...,...,...,...,...,...,...,...,...,...
18062036,2660,Hy-vee Food Store,1951 S MAIN AVE,Sioux Center,51250.0,"(-96.18, 43.06)",84.0,Sioux,1
18874425,2667,Hy-vee Mainstreet,2611 PIERCE ST,Sioux City,51104,"(-96.41, 42.52)",97.0,Woodbury,1
20660328,2717,Hy-vee Gas 5018,636 Lincoln Way,Ames,50010,"(-93.62, 42.02)",85.0,Story,1
20678088,2682,Hy-vee Wdm Health Market,375 South Jordon Creek Parkway,West Des Moines,50266.0,"(-93.81, 41.56)",25.0,Dallas,1


# Transaction Master Table

In [65]:
transaction_col = ['Date',
                   'VendorNumber',
                   'StoreNumber',
                   'ItemNumber',
                   'BottlesSold',
                   'VolumeSold(Gallons)',
                   'Sale(Dollars)']

transaction = sales[transaction_col].copy()

transaction

,Date,VendorNumber,StoreNumber,ItemNumber,BottlesSold,VolumeSold(Gallons),Sale(Dollars)
0,03/16/2016,330.0,3762,80457,1,0.26,10.85
1,07/15/2019,421.0,4320,10008,6,2.77,94.50
2,07/01/2020,370.0,2643,15628,2,0.92,97.44
3,07/11/2019,421.0,3456,10009,2,0.52,20.22
4,07/23/2019,260.0,5151,100107,6,2.93,162.00
...,...,...,...,...,...,...,...
21641152,06/29/2021,370.0,5423,34007,12,3.17,269.88
21641153,06/30/2021,260.0,5146,72928,12,2.37,163.08
21641154,06/30/2021,259.0,2521,17127,12,3.17,90.00
21641155,06/29/2021,260.0,4014,43337,12,3.17,211.56


In [66]:
transaction['Date'] = pd.to_datetime(transaction['Date'])

In [74]:
transaction.rename(columns = {'VolumeSold(Gallons)': 'VolumeSold', 'Sale(Dollars)': 'Sale'}, inplace = True)

# Set Up SQLite Database

In [112]:
con = sqlite3.connect('IowaLiquor.db')

In [113]:
cur = con.cursor()

In [114]:
cur.execute('''CREATE TABLE Product
                (ItemNumber, ItemDescription, Category, CategoryName)''')

cur.execute('''CREATE TABLE Price
                (Date, ItemNumber, StateBottleRetail, BottlesSold, Sale)''')

cur.execute('''CREATE TABLE Venor
                (VendorNumber, VendorName)''')

cur.execute('''CREATE TABLE Store
                (StoreNumber, StoreName, Address, City, ZipCode, CountyNumber, County)''')

cur.execute('''CREATE TABLE Transactions
                (Date, VendorNumber, StoreNumber, ItemNumber, BottlesSold, VolumeSold, Sale)''')

In [115]:
product.to_sql('Product', con, if_exists = 'append', index = False)
product_price.to_sql('Price', con, if_exists = 'append', index = False)
vendor.to_sql('Vendor', con, if_exists = 'append', index = False)
store_sql.to_sql('Store', con, if_exists = 'append', index = False)
transaction.to_sql('Transactions', con, if_exists = 'append', index = False)

In [116]:
pd.read_sql('''SELECT * FROM Product''', con)

,ItemNumber,ItemDescription,Category,CategoryName
0,80457,Ryans Cream Liqueur,1081200.0,cream liqueurs
1,10008,Scoresby Rare Scotch,1012200.0,scotch whiskies
2,15628,Jameson,1012400.0,irish whiskies
3,10009,Scoresby Rare Scotch,1012200.0,scotch whiskies
4,100107,Captain Morgan OSR Glass w/50ml Apple Smash & ...,1701100.0,temporary & specialty packages
...,...,...,...,...
15337,47647,Branson VSOP Royal,1052100.0,imported brandies
15338,65198,Tequila Rose Strawberry,1081200.0,cream liqueurs
15339,12853,Rich & Rare Reserve Mini,1012100.0,canadian whiskies
15340,976409,Ole Smoky Sour Watermelon Moonshine,1091300.0,neutral grain spirits flavored


In [117]:
con.commit()

In [ ]:
#con.close()

# Data Analysis

### Vendor Analysis

How many product does a vendor sell?

In [138]:
# I'll look at Jim Beam
vendor

,VendorNumber,VendorName
0,330.0,Gemini Spirits
1,421.0,SAZERAC COMPANY INC
2,370.0,PERNOD RICARD USA
4,260.0,DIAGEO AMERICAS
7,410.0,Patron Spirits Company
8,65.0,Jim Beam Brands
10,115.0,CONSTELLATION BRANDS INC
11,434.0,LUXCO INC
12,259.0,Heaven Hill Brands
14,626.0,JDSO INC / Red Boot Distillery


In [119]:
JimBeam1 = pd.read_sql('''SELECT VendorNumber, ItemNumber 
                FROM Transactions
                WHERE VendorNumber = 65.0
                GROUP BY VendorNumber, ItemNumber''', con)

In [121]:
JimBeam2 = JimBeam1.merge(product[['ItemNumber', 'Category']], on = 'ItemNumber', how = 'left')

In [178]:
product.shape

(15342, 4)

In [123]:
JimBeam1

,VendorNumber,ItemNumber
0,65.0,155
1,65.0,168
2,65.0,173
3,65.0,237
4,65.0,399
...,...,...
1089,65.0,927080
1090,65.0,936002
1091,65.0,944561
1092,65.0,954907


In [181]:
product_test2 = product.copy()

In [184]:
product_test2.shape

(15342, 4)

In [185]:
product_test2.dropna(inplace = True)

In [186]:
product_test2.isna().sum(axis = 0)

ItemNumber         0
ItemDescription    0
Category           0
CategoryName       0
dtype: int64

In [187]:
product_test2.shape

(14937, 4)

In [180]:
product.isna().sum(axis = 0)

ItemNumber           0
ItemDescription      0
Category           401
CategoryName       405
dtype: int64

In [124]:
JimBeam2

,VendorNumber,ItemNumber,Category
0,65.0,155,NaN
1,65.0,155,1701200.0
2,65.0,168,1701100.0
3,65.0,168,1032200.0
4,65.0,168,NaN
...,...,...,...
1332,65.0,927080,NaN
1333,65.0,936002,1032200.0
1334,65.0,944561,NaN
1335,65.0,954907,1051100.0


# Store Analysis

In [125]:
store_HyVee = store.copy()

In [126]:
store_HyVee['HyVee_control'] = np.where(store_HyVee['StoreName'].str.contains('Hy-vee'),1,0)

In [128]:
store_HyVee = store_HyVee.loc[store_HyVee['HyVee_control'] == 1]

In [129]:
store_HyVee.head()

,StoreNumber,StoreName,Address,City,ZipCode,StoreLocation,CountyNumber,County,HyVee_control
9,2663,Hy-vee Food Store,8701 Douglas,Urbandale,50322.0,"[-93.74, 41.63]",77.0,Polk,1
18,2647,Hy-vee 7,5050 Edgewood Rd,Cedar Rapids,52411.0,"[-91.7, 42.03]",57.0,Linn,1
34,2565,Hy-vee Food Store 1636,819 N Grand Ave,Spencer,51301.0,"[-95.15, 43.15]",21.0,Clay,1
61,2662,Hy-vee Wine & Spirits,"522 Mulberry, Suite A",Muscatine,52761.0,"[-91.04, 41.43]",70.0,Muscatine,1
64,2555,Hy-vee Food Store,3111 Main,Keokuk,52632.0,"[-91.4, 40.42]",56.0,Lee,1


In [145]:
County_Pop = pd.read_csv('Iowa_County_Population.csv')

In [146]:
County_Pop['Population'] = County_Pop['Population'].str.replace(',','').astype(int)

In [148]:
County_Pop.head()

,Rank,County,Population
0,1,Polk,479612
1,2,Linn,223861
2,3,Scott,172446
3,4,Johnson,148577
4,5,Black Hawk,132393


In [149]:
County_Merge = County_Pop.merge(store_HyVee[['County', 'StoreName']], on = 'County', how = 'left')

In [153]:
County_Merge

,Rank,County,Population,StoreName
0,1,Polk,479612,Hy-vee Food Store
1,1,Polk,479612,Hy-vee Wine & Spirits
2,1,Polk,479612,Hy-vee 6
3,1,Polk,479612,Hy-vee 4
4,1,Polk,479612,Hy-vee
...,...,...,...,...
781,98,Ringgold,4964,Hy-vee Food Store
782,98,Ringgold,4964,Hy-vee Food Store
783,98,Ringgold,4964,Hy-vee Food Store
784,98,Ringgold,4964,Hy-vee Food Store


In [155]:
County_Pivot = County_Merge.groupby(['County']).agg({'Population': 'mean', 'StoreName': 'count'})

In [156]:
County_Pivot['Pop_per_Store'] = County_Pivot['Population'] / County_Pivot['StoreName']

In [158]:
County_Pivot.sort_values(by = ['Pop_per_Store'])

,Population,StoreName,Pop_per_Store
County,,,
Wayne,6429,10,6.429000e+02
Ringgold,4964,7,7.091429e+02
Taylor,6167,6,1.027833e+03
Page,15268,13,1.174462e+03
Appanoose,12452,10,1.245200e+03
Monroe,7807,6,1.301167e+03
Cass,13091,10,1.309100e+03
Decatur,7979,6,1.329833e+03
Clarke,9360,7,1.337143e+03


# Product Analysis

In [159]:
product_aggcat = product.copy()

In [160]:
def agg_cat(name):
    name = str(name)
    name = re.sub('Neutral Grain Spirits.*','Neutral Grain Spirits',name)
    name = re.sub(r'Temporary.*Specialty','Temporary & Specialty',name)
    name = re.sub(r'Cocktails.*RTD','Cocktails / RTD',name)
    
    if re.findall(r'\bLiqueurs?\b', name):
        name = 'Liqueurs'
    elif re.findall(r'Schnapps', name):
        name = 'Liqueurs'
    elif re.findall(r'Triple Sec', name):
        name = 'Liqueurs'
    elif re.findall(r'Brand(y\b|ies\b)', name):
        name = 'Brandies'
    elif re.findall(r'Bourbon', name):
        name = 'Whiskies'
    elif re.findall(r'Single Malt Scotch', name):
        name = 'Whiskies'
    elif re.findall(r'(Whisk)(ies|y)', name):
        name = 'Whiskies'
    elif re.findall(r'\bVodkas?\b', name):
        name = 'Vodka'  
    elif re.findall(r'\bRums?\b', name):
        name = 'Rum'
    elif re.findall(r'Tequila', name):
        name = 'Tequila'
    elif re.findall(r'Mezcal', name):
        name = 'Tequila'    
    elif re.findall(r'Gin', name):
        name = 'Gin'
    elif re.findall(r'Cocktails / RTD', name):
        name = 'Specialty'
    elif re.findall(r'Distilled \bSpirits?\b Specialty', name):
        name = 'Specialty'
    elif re.findall(r'Temporary & Specialty Packages', name):
        name = 'Specialty'
    elif re.findall(r'Special Order Items', name):
        name = 'Specialty'
    elif re.findall(r'Holiday VAP', name):
        name = 'Specialty'
    elif re.findall(r'Neutral Grain Spirits', name):
        name = 'Specialty'
    elif re.findall(r'Delisted', name):
        name = 'Delisted'
    return name

In [161]:
product_aggcat['CategoryName'] = product_aggcat['CategoryName'].map(agg_cat)
product_aggcat['CategoryName'].value_counts().sort_values(ascending = False)

special order items                     945
temporary & specialty packages          883
american flavored vodka                 873
straight bourbon whiskies               819
single malt scotch                      645
scotch whiskies                         531
imported flavored vodka                 515
100% agave tequila                      471
american vodkas                         449
mixto tequila                           415
blended whiskies                        414
cocktails /rtd                          413
nan                                     405
misc. imported cordials & liqueurs      400
canadian whiskies                       391
american cordials & liqueur             326
imported vodka                          310
aged dark rum                           306
white rum                               251
flavored rum                            249
american schnapps                       234
cream liqueurs                          233
american dry gins               